In [2]:
import random

class Sentiment: ## made a class named sentiment which has var for positive, negative and neutral stored.
    NEGATIVE="NEGATIVE"
    NEUTRAL="NEUTRAL"
    POSITIVE="POSTIVE"
    
class Review: ## made a data class to make it easy to access the review and the score given with .text nad .score
    def __init__(self,text,score):
        self.text=text
        self.score=score
        self.sentiment=self.get_sentiments()
        
        
    def get_sentiments(self): ## this will tell us the sentiments on stars/score given 
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score ==3:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE
        
        
class ReviewContainer:
    def __init__(self,reviews):
        self.reviews=reviews
        
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]


    def evenly_distributed(self):
        negative= list(filter(lambda x : x.sentiment==Sentiment.NEGATIVE,self.reviews)) ##we are keeping track of the all the negative comments we can do the same thing for positive
    
        positive=list(filter(lambda x : x.sentiment==Sentiment.POSITIVE,self.reviews))
        
        positive_shrunk=positive[:len(negative)]
        self.reviews=negative+positive_shrunk ##we are just shrunking the amount of reviews storing which only has positive and negative which are equal to the amount of negative
        
        random.shuffle(self.reviews)
        
               
            
        
        
        
        
        

In [3]:
import json
file_name='./Books_small_10000.json' ## saved the review file as file_name

reviews=[] ## this is an empty array
with open(file_name)as f: ##open the file and now its called f
    for line in f:
       review=json.loads(line) ##at this line we loaded the file with json package
       reviews.append(Review(review['reviewText'], review['overall'])) ## we add all the reviews in thie empty array
reviews[5].sentiment

'POSTIVE'

#### PREP DATA

In [4]:
## Split our 1000 reviews in traning and test data
from sklearn.model_selection import train_test_split
training,test=train_test_split(reviews,test_size=0.33,random_state=42)##At this stage we are splittinh our 1000 review in test and training sets 
train_container= ReviewContainer(training)
test_container= ReviewContainer(test)

##this will return x and y currently which are training as x and test as y test_size is 0.33 

In [5]:
train_container.evenly_distributed()
train_x=train_container.get_text() ##first we splited our training data into x and y.we use file comprehension to just get the text we did this
train_y=train_container.get_sentiment() ##and for train_y which has only sentiment 

test_container.evenly_distributed()
test_x=test_container.get_text() ##same as trainig set we split and did file comprehension
test_y=test_container.get_sentiment()

print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGATIVE)) 
##now we have equal number of negative and positive sentiments



436
436


## Bag of words vectorization

In [6]:
##now we will fit our bag_of_words model to the training set and then we will build a classifier on that training set
##in order to perform machine learning on text documents we first need to turn text content into numerical feature vectors.
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

vectorizer=TfidfVectorizer() ## this method will turn your text intup into numerical feature vectors check more about bag of words on google.
train_x_vectors=vectorizer.fit_transform(train_x) ##fit_transform needs the train_x var which has all the text data.
test_x_vectors=vectorizer.transform(test_x)


#### Classification

##### Linear SVM

In [7]:
from sklearn import svm

clf_svm=svm.SVC(kernel='linear')
clf_svm.fit(train_x_vectors,train_y) ##Fitting the classifier to the train x any y set

test_x[0]
clf_svm.predict(test_x_vectors[0])

array(['POSTIVE'], dtype='<U8')

In [8]:
clf_svm.score(test_x_vectors,test_y)##82% Accuracy is the score of the analysis.

0.8076923076923077

### we equally distributed the numbers of positive nad negative comments an then checked the F1 score  

In [9]:
##F1 SCORES
from sklearn.metrics import f1_score
f1_score(test_y,clf_svm.predict(test_x_vectors),average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE])



array([0.80582524, 0.80952381])

In [10]:
test_set=['I love you','bad book dont buy it','horrible waste of time']
new_test=vectorizer.transform(test_set) ##we transformed the test_set here

clf_svm.predict(new_test) ##and then we predicted the sentiments


array(['POSTIVE', 'NEGATIVE', 'NEGATIVE'], dtype='<U8')


#### Saving your model

In [11]:
import pickle
with open('./Models/sentiment_classifier.pkl', 'wb') as f:
    pickle.dump(clf_svm,f)

#### Load Model

In [12]:
with open('./Models/sentiment_classifier.pkl', 'rb') as f:
    loaded_clf=pickle.load(f)

In [14]:
print(test_x[0])
loaded_clf.predict(test_x_vectors[0])

I enjoyed this book very much.  The plot was interesting and the characters were persons one could identify with.  Hard to put down once you start reading.


array(['POSTIVE'], dtype='<U8')